In [2]:
import numpy as np 
import pandas as pd 
import re

from time import sleep
import time as time


In [3]:
# 파일주소는 맞게 바꿔주세요 
sample_data = pd.read_csv('news_headline_total.csv' ,header=None)
sample_data

,0,1,2,3,4,5,6,7,8,9,...,24965,24966,24967,24968,24969,24970,24971,24972,24973,24974
0,"[아이넷 AI 로봇 기자] 특징주, 무림페이퍼-제지 테마 상승세에 10.4% ↑","[아이넷 AI 로봇 기자] 특징주, 영풍제지-골판지 제조 테마 상승세에 28.29% ↑","[아이넷 AI 로봇 기자] 특징주, 깨끗한나라-제지 테마 상승세에 5.65% ↑","[아이넷 AI 로봇 기자] 특징주, 신풍제지-제지 테마 상승세에 20.43% ↑","""한섬, 업황 회복으로 실적 모멘텀 지속""...비중확대는 DSR무관 야호스탁 상품으로","[아이넷 AI 로봇 기자] 특징주, 무림SP-제지 테마 상승세에 7.85% ↑",데브시스터즈 1분기 영업이익 94억원…작년 동기 대비 60.2%↓,"""호텔·상가 리츠, 리오프닝 기대""...대비는 DSR무관 골드스탁 재원으로","파멥신, 제프리스 헬스케어 컨퍼런스 초청 참석","한화자산운용, 세계 최초 대체자산 전반에 투자하는 ETF 상장",...,"[코스닥] 6.19p(0.59%) 내린 1,043.64(장종료)","[레이더M] F&F와 손잡은 센트로이드, 테일러메이드 거래 종결 박차","[아이넷 AI 로봇 기자] 특징주, 우정바이오-코로나19(음압병실/음압구급차) 테마...","[아이넷 AI 로봇 기자] 한진중공업(097230), 도마변동9재정비촉진구역 주택재...","[레이더M] 교직원공제회, 국내 PEF 블라인드 펀드 총 8개사 선정",[아이넷 AI 로봇 기자] 디오 '디지털 임플란트는 디오!' 증권사 발표에 강세…3.8%,"[아이넷 AI 로봇 기자] 에스트래픽(234300), 인천국제공항공사와 71.5억원...","미래에셋자산운용, 글로벌 테마형 ETF 신규 상장 이벤트","메디아나, 2분기 영업익 18.5억원…2분기 연속 흑자 행진","두산중공업, 미 뉴스케일파워에 추가 투자…소형모듈원전 사업 협력 강화"


In [176]:
# T 전치행렬 곱해서  가로 세로 변경 
news_title= sample_data.T
news_title.columns = ['title'] 

# 모든라벨을 먼저 중립인 3으로 표시  
news_title['label'] = 3 

In [177]:
# 화살표 의미 데이터 전처리 
news_title = news_title.replace(['↑\s*', '↓\s*']  ,  [' 상승 ' , ' 하락 '], regex=True)


In [178]:
#  상관없는 뉴스 타이틀 삭제 
drop_idx = news_title.index[(news_title['title'].str.contains('아이넷')) | (news_title['title'].str.contains(r'[[표]]')) | (news_title['title'].str.contains(r'[[부고]]'))
                           |(news_title['title'].str.contains(r'[[게시판]]')) | (news_title['title'].str.contains(r'[[증시신상품]]')) | (news_title['title'].str.contains(r'[[유가증권시장 공시]]'))
                           |(news_title['title'].str.contains(r'[[M&A 매물장터]]')) | (news_title['title'].str.contains(r'[[인사]]')) | (news_title['title'].str.contains(r'[[포토]]')) |(news_title['title'].str.contains(r'[[공모주 인사이드]]'))
                           |(news_title['title'].str.contains(r'[[장마감 이슈]]')) | (news_title['title'].str.contains(r'[[AI 개장전 분석]]')) | (news_title['title'].str.contains('오늘의')) | (news_title['title'].str.contains('이 상품'))
                           ]

news_title = news_title.drop(drop_idx)

In [179]:
# 라벨 작업 (상승, 오른 , 흑자 , 최대 실적  , 최대실적 , 순매수 , 매수 , 고성장 , 반등 , 기대 , 증가  , (영업이익 늘어 2개 포함)
up_stock_word = news_title.index[(news_title['title'].str.contains('상승')) | (news_title['title'].str.contains('오른'))| (news_title['title'].str.contains('흑자'))
                        | (news_title['title'].str.contains('최대실적')) | (news_title['title'].str.contains('최대 실적'))| (news_title['title'].str.contains('순매수'))
                        | (news_title['title'].str.contains('매수')) | (news_title['title'].str.contains('고성장'))| (news_title['title'].str.contains('반등'))| (news_title['title'].str.contains('기대'))
                        | (news_title['title'].str.contains('증가')) | (news_title['title'].str.contains('선방'))  | (news_title['title'].str.contains('상향'))| (news_title['title'].str.contains('역대'))  
                        | (news_title['title'].str.contains('최대')) | (news_title['title'].str.contains('긍정적'))| (news_title['title'].str.contains('낙관'))
                        | (news_title['title'].str.contains('깜짝실적'))].values
for i in up_stock_word : 
    news_title['label'][i] = 1 
    

C:\Users\crid2\AppData\Local\Temp\ipykernel_1288\587874182.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_title['label'][i] = 1


In [180]:
# 폭락, 기대이하 감소 , 하락
down_stock_word = news_title.index[(news_title['title'].str.contains('하락')) | (news_title['title'].str.contains('내린'))| (news_title['title'].str.contains('적자'))
                        | (news_title['title'].str.contains('최저'))| (news_title['title'].str.contains('우려'))| (news_title['title'].str.contains('순매도'))
                        | (news_title['title'].str.contains('공매도'))| (news_title['title'].str.contains('저성장'))| (news_title['title'].str.contains('매도'))
                        | (news_title['title'].str.contains('감소'))| (news_title['title'].str.contains('붕괴')) 
                        | (news_title['title'].str.contains('잃었다')) |  (news_title['title'].str.contains('증발'))  |  (news_title['title'].str.contains('부정적'))                                          
                        | (news_title['title'].str.contains('하향')) | (news_title['title'].str.contains('뒷걸음')) | (news_title['title'].str.contains('폭락'))| (news_title['title'].str.contains('경고'))
                        | (news_title['title'].str.contains('반등 실패')) | (news_title['title'].str.contains('부진'))].values
for i in down_stock_word : 
    news_title['label'][i] = 0 

C:\Users\crid2\AppData\Local\Temp\ipykernel_1288\2981977823.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_title['label'][i] = 0


In [181]:
# 중립 저가매수 , 성장주 , 테마주 
noresult_stock_word = news_title.index[(news_title['title'].str.contains('저가매수'))|(news_title['title'].str.contains('전망'))
                                      ].values
for i in noresult_stock_word : 
    news_title['label'][i] = 3 

C:\Users\crid2\AppData\Local\Temp\ipykernel_1288\149451337.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_title['label'][i] = 3


In [182]:
news_title = news_title.reset_index(drop=True)

In [183]:
news_title

,title,label
0,"""한섬, 업황 회복으로 실적 모멘텀 지속""...비중확대는 DSR무관 야호스탁 상품으로",3
1,데브시스터즈 1분기 영업이익 94억원…작년 동기 대비 60.2% 하락,0
2,"""호텔·상가 리츠, 리오프닝 기대""...대비는 DSR무관 골드스탁 재원으로",1
3,"파멥신, 제프리스 헬스케어 컨퍼런스 초청 참석",3
4,"한화자산운용, 세계 최초 대체자산 전반에 투자하는 ETF 상장",3
...,...,...
11046,"[코스피] 11.34p(0.35%) 내린 3,232.70(장종료)",0
11047,"[코스닥] 6.19p(0.59%) 내린 1,043.64(장종료)",0
11048,"미래에셋자산운용, 글로벌 테마형 ETF 신규 상장 이벤트",3
11049,"메디아나, 2분기 영업익 18.5억원…2분기 연속 흑자 행진",1


In [184]:
news_title.groupby(by='label').count()

,title
label,
0,1994
1,3029
3,6028


In [185]:
# 전처리한 데이터 csv 파일로 저장 
news_title.to_csv('22_news_title.csv')